In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install pydicom
!pip install python-gdcm
!pip install pylibjpeg
!pip install ipyvolume
!pip install ipywidgets
!jupyter nbextension enable --py --sys-prefix widgetsnbextension

import matplotlib.pyplot as plt
import pydicom
from tqdm.notebook import tqdm
import ipyvolume as ipv

In [ ]:
os.chdir('/kaggle/input/rsna-2022-cervical-spine-fracture-detection')

In [ ]:
df = pd.read_csv('train.csv')

In [ ]:
df.head()

In [ ]:
def viz(img):
    pixels = []
    folder = os.listdir(os.path.join('train_images',img))
    folder.sort(key=lambda x: int(x.split('.')[0]))
    for i in folder:
#         print(i)
        p = os.path.join(os.path.join('train_images',img),i)
#         print(p)
        ds = pydicom.dcmread(p)
#         print(ds.pixel_array)
        pixels.append(ds.pixel_array)
#         plt.imshow(ds.pixel_array) 
#         plt.show()
    return pixels

In [ ]:
pix = viz('1.2.826.0.1.3680043.6200')

In [ ]:
fig = ipv.figure()
scatter = ipv.examples.gaussian(show=False)


In [ ]:
ipv.pylab.volshow(np.array(pix))

In [ ]:
np.random.seed(19680801)


class IndexTracker:
    def __init__(self, ax, X):
        self.ax = ax
        ax.set_title('use scroll wheel to navigate images')

        self.X = X
        rows, cols, self.slices = X.shape
        self.ind = self.slices//2

        self.im = ax.imshow(self.X[:, :, self.ind])
        self.update()

    def on_scroll(self, event):
        print("%s %s" % (event.button, event.step))
        if event.button == 'up':
            self.ind = (self.ind + 1) % self.slices
        else:
            self.ind = (self.ind - 1) % self.slices
        self.update()

    def update(self):
        self.im.set_data(self.X[:, :, self.ind])
        self.ax.set_ylabel('slice %s' % self.ind)
        self.im.axes.figure.canvas.draw()


fig, ax = plt.subplots(1, 1)

X = np.random.rand(20, 20, 40)

tracker = IndexTracker(ax, np.array(pix))


fig.canvas.mpl_connect('scroll_event', tracker.on_scroll)
plt.show()

In [ ]:
class ImageData(Dataset):
    
    def __init__(self,df,transform=None):
        self.df = df
        self.transform = transform
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self,index):
        uid = self.df.iloc[index,0]
        folder = os.listdir(os.path.join('train_images',uid))
        folder.sort(key=lambda x: int(x.split('.')[0]))
        for slices in folder:
            slice_path = os.path.join(os.path.join('train_images',uid),slices)
            ds = pydicom.dcmread(slice_path)
            vol.append(ds.pixel_array)
        image = np.array(vol)    
        labels = torch.tensor(self.df.iloc[index,1],dtype=torch.long)
        if self.transform is not None:
            image = self.transform(image)
        return image, labels

transform = transforms.Compose([transforms.Resize(size = (224,224)),transforms.ToTensor(), transforms.Normalize(mean = [0.485, 0.456, 0.406],std = [0.229, 0.224, 0.225])])#

In [ ]:
train_generator = DataLoader(ImageData(df_train,transform=transform),batch_size = 100,shuffle = True)
val_generator = DataLoader(ImageData(df_val,transform=transform),batch_size = 100,shuffle = True)
test_generator = DataLoader(ImageData(df_test,transform=transform),batch_size = 100,shuffle = True)

In [ ]:
def viz(img):
    pixels = []
    folder = os.listdir(os.path.join('train_images',img))
    folder.sort(key=lambda x: int(x.split('.')[0]))
    for i in folder:
#         print(i)
        p = os.path.join(os.path.join('train_images',img),i)
#         print(p)
        ds = pydicom.dcmread(p)
#         print(ds.pixel_array)
        pixels.append(ds.pixel_array)
#         plt.imshow(ds.pixel_array) 
#         plt.show()
    return pixels